In [1]:
from __future__ import print_function
from PIL import Image
import numpy as np
import tensorflow as tf
import os
import glob
import matplotlib.pyplot as plt

layers = tf.contrib.layers

# 1.model

In [2]:
# def conv_block(inputs, out_channels, name='conv'):
#     with tf.variable_scope(name):
#         conv = tf.layers.conv2d(inputs, out_channels, kernel_size=3, padding='SAME')
#         conv = tf.contrib.layers.batch_norm(conv, updates_collections=None, decay=0.99, scale=True, center=True)
#         conv = tf.nn.relu(conv)
#         conv = tf.contrib.layers.max_pool2d(conv, 2)
#         return conv

In [3]:
# def encoder(x, h_dim, z_dim, reuse=False):
#     with tf.variable_scope('encoder', reuse=reuse):
#         net = conv_block(x, h_dim, name='conv_1')
#         net = conv_block(net, h_dim, name='conv_2')
#         net = conv_block(net, h_dim, name='conv_3')
#         net = conv_block(net, z_dim, name='conv_4')
#         net = tf.contrib.layers.flatten(net)
#         return net

In [4]:
def encoder(x, h_dim, z_dim, reuse=False):
    with tf.variable_scope("encoder", reuse=reuse):
        net = layers.conv2d(x, h_dim, (4,4), 2)
        net = tf.contrib.layers.batch_norm(net, updates_collections=None, decay=0.999, scale=True, center=True)
        net = layers.conv2d(net, h_dim, (4,4), 2)
        net = tf.contrib.layers.batch_norm(net, updates_collections=None, decay=0.99, scale=True, center=True)
        net = layers.conv2d(net, h_dim, (4,4), 2)
        net = tf.contrib.layers.batch_norm(net, updates_collections=None, decay=0.99, scale=True, center=True)
        net = layers.conv2d(net, z_dim, (4,4), 2)
        net = tf.contrib.layers.batch_norm(net, updates_collections=None, decay=0.99, scale=True, center=True)
        net = layers.flatten(net)
        return net

# 2.Loss function

In [5]:
def bregman_distortion_loss(emb_q, emb_x, y):
    def euclidean_distance(a, b):
        # a.shape = N x D -> embeddings for N query
        # b.shape = M x D -> prototypical embeddings for each class
        N, D = tf.shape(a)[0], tf.shape(a)[1]
        M = tf.shape(b)[0]
        a = tf.expand_dims(a, axis=1) #a:Nx1xD
        a = tf.tile(a, (1, M, 1)) #a:NxMxD
        b = tf.expand_dims(b, axis=0) #b:1xMxD
        b = tf.tile(b, (N, 1, 1)) #b:NxMxD
        return tf.reduce_mean(tf.square(a - b), axis=2), N, M #NxM each column k means mse(b_k, a)
    dists, N, M = euclidean_distance(emb_q, emb_x)
    num_classes = M
    num_query = N//M
    y_one_hot = tf.one_hot(y, depth=num_classes)
    log_p_y = tf.reshape(tf.nn.log_softmax(-dists), [num_classes, num_query, -1])
    ce_loss = -tf.reduce_mean(tf.reshape(tf.reduce_sum(tf.multiply(y_one_hot, log_p_y), axis=-1), [-1]))
    acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(log_p_y, axis=-1), y)))
    return ce_loss, acc

# 3.PARAMETER

In [6]:
n_epochs = 100
n_episodes = 100
n_way = 20
n_shot = 5
n_query = 15
n_examples = 350
im_width, im_height, channels = 84, 84, 3
h_dim = 64
z_dim = 64

# 4.BUILD GRAPH

In [7]:
x = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels]) #support(shot) images
q = tf.placeholder(tf.float32, [None, None, im_height, im_width, channels]) #query images
x_shape = tf.shape(x)
q_shape = tf.shape(q)
num_classes, num_support = x_shape[0], x_shape[1]
num_queries = q_shape[1]
y = tf.placeholder(tf.int64, [None, None]) #label : num_classes x num_queries matrix

emb_x = encoder(tf.reshape(x, [num_classes * num_support, im_height, im_width, channels]), h_dim, z_dim)
emb_dim = tf.shape(emb_x)[-1]
emb_x = tf.reduce_mean(tf.reshape(emb_x, [num_classes, num_support, emb_dim]), axis=1) #prototical embeddings
emb_q = encoder(tf.reshape(q, [num_classes * num_queries, im_height, im_width, channels]), h_dim, z_dim, reuse=True) #query embeddings

In [8]:
ce_loss, acc = bregman_distortion_loss(emb_q, emb_x, y)
train_op = tf.train.AdamOptimizer().minimize(ce_loss)

In [9]:
sess = tf.InteractiveSession()
init_op = tf.global_variables_initializer()
sess.run(init_op)

# 5.TRAIN SESSION

In [10]:
# Load Train Dataset
data_generator_path = os.environ['DATA_GENERATOR']
train_split_path = os.path.join(data_generator_path, "datasets/mini-imagenet/npy", "mini-imagenet-train.npy")
train_dataset = np.load(train_split_path)
n_classes = train_dataset.shape[0]
print(train_dataset.shape)

(64, 350, 84, 84, 3)


In [12]:
for ep in range(n_epochs):
    for epi in range(n_episodes):
        epi_classes = np.random.permutation(n_classes)[:n_way]
        support = np.zeros([n_way, n_shot, im_height, im_width, channels], dtype=np.float32)
        query = np.zeros([n_way, n_query, im_height, im_width, channels], dtype=np.float32)
        for i, epi_cls in enumerate(epi_classes):
            selected = np.random.permutation(n_examples)[:n_shot + n_query]
            support[i] = train_dataset[epi_cls, selected[:n_shot]]
            query[i] = train_dataset[epi_cls, selected[n_shot:]]
#         support = np.expand_dims(support, axis=-1)
#         query = np.expand_dims(query, axis=-1)
        labels = np.tile(np.arange(n_way)[:, np.newaxis], (1, n_query)).astype(np.uint8)
        _, ls, ac = sess.run([train_op, ce_loss, acc], feed_dict={x: support, q: query, y:labels})
        if (epi+1) % 50 == 0:
            print('[epoch {}/{}, episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(ep+1, n_epochs, epi+1, n_episodes, ls, ac))

[epoch 1/100, episode 50/100] => loss: 2.99573, acc: 0.05000
[epoch 1/100, episode 100/100] => loss: 2.99573, acc: 0.05000


KeyboardInterrupt: 

# 6.EVAL_SESSION

In [ ]:
# Load Test Dataset
data_generator_path = os.environ['DATA_GENERATOR']
train_split_path = os.path.join(data_generator_path, "datasets/mini-imagenet/npy", "mini-imagenet-test.npy")
train_dataset = np.load(train_split_path)
n_classes = train_dataset.shape[0]
print(train_dataset.shape)

In [ ]:
n_test_episodes = 600
n_test_way = 5
n_test_shot = 5
n_test_query = 15

In [ ]:
print('Testing...')
avg_acc = 0.
for epi in range(n_test_episodes):
    epi_classes = np.random.permutation(n_test_classes)[:n_test_way]
    support = np.zeros([n_test_way, n_test_shot, im_height, im_width, channels], dtype=np.float32)
    query = np.zeros([n_test_way, n_test_query, im_height, im_width, channels], dtype=np.float32)
    for i, epi_cls in enumerate(epi_classes):
        selected = np.random.permutation(n_examples)[:n_test_shot + n_test_query]
        support[i] = test_dataset[epi_cls, selected[:n_test_shot]]
        query[i] = test_dataset[epi_cls, selected[n_test_shot:]]
    # support = np.expand_dims(support, axis=-1)
    # query = np.expand_dims(query, axis=-1)
    labels = np.tile(np.arange(n_test_way)[:, np.newaxis], (1, n_test_query)).astype(np.uint8)
    ls, ac = sess.run([ce_loss, acc], feed_dict={x: support, q: query, y:labels})
    avg_acc += ac
    if (epi+1) % 50 == 0:
        print('[test episode {}/{}] => loss: {:.5f}, acc: {:.5f}'.format(epi+1, n_test_episodes, ls, ac))
avg_acc /= n_test_episodes
print('Average Test Accuracy: {:.5f}'.format(avg_acc))